# French to English Translator using Deep Learning
This project aims to build a French to English translator using an artificial Recurrent Neural Network (RNN) called Long Short-Term Memory (LSTM).

In [ ]:
%%capture
# install Tensorflow
!pip install tensorflow 

# Loading spacy's French and English languages 
!pip install -U spacy
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm

In [ ]:
# Import required packages
import numpy as np
import pandas as pd
import fr_core_news_sm
import en_core_web_sm
import tensorflow as tf
tf.__version__

'2.4.1'

### Importing data 

The data comes from a `.txt` file containing more than 160000 sentences with their translation separated by a tab (`\t`).

The data can be found on this link: https://go.aws/38ECHUB

For performance purposes, we will not take the whole dataset but a sample of 5000 sentences instead. This will allow us to faster iterate and avoid bugs related to our need for computer power.

In [ ]:
# Loading data
doc = pd.read_csv("https://go.aws/38ECHUB", delimiter="\t", header=None)
doc.head()

,0,1
0,Go.,Va !
1,Hi.,Salut !
2,Run!,Cours !
3,Run!,Courez !
4,Wow!,Ça alors !


In [ ]:
doc.shape

(160538, 2)

In [ ]:
# Let's just take a sample of 5000 sentences to avoid slowness
doc = doc.sample(5000)

In [ ]:
# Loading of the entire corpus of French and English sentences
fr_corpus = " ".join(doc.iloc[:, 1].to_list())
en_corpus = " ".join(doc.iloc[:, 0].to_list())

### Preprocessing 

The main purpose of the preprocessing step is to express each French entry sentence in a sequence of clues.

i.e.:

* I'm sick ---> $[123, 21, 34, 0, 0, 0, 0, 0]$

This gives a *shape* -> `(batch_size, max_len_of_a_sentence)`.

The clues correspond to a number that we will have to assign for each word token. 

The zeros correspond to what are called [*padded_sequences*](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) which allow all word sequences to have the same length (mandatory for our algorithm). 

The transformation of our target sentences will not be exactly the same as that of our input sentences. In addition to all the steps we will have performed for the input sentences, we will also have to *categorize* our target sentences. In other words, an example tensor would look like : 

* I am sick ---> $\begin{bmatrix} 1&0&0&...&0&0 \\ 0&0&0&...&1&0 \\ ... \\ 0&1&0&...&0&0 \end{bmatrix}$

This gives a *shape* -> `(batch_size, max_len_of_an_english_sentence, num_of_classes)`.

To do this, we are going to use : 

* `Spacy` for Tokenization 
* `Tensorflow` for [padded_sequence](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) & [categorization](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical?hl=en)

In [ ]:
# Loading both corpora into spacy 
nlp_fr = fr_core_news_sm.load()
nlp_fr.max_length = len(fr_corpus)

nlp_en = en_core_web_sm.load()
nlp_en.max_length = len(en_corpus)

fr_doc = nlp_fr(fr_corpus)
en_doc = nlp_en(en_corpus)

In [ ]:
# Tokenization of each sentence via spacy 
doc["fr_tokens"] = doc.iloc[:, 1].apply(nlp_fr.tokenizer)
doc["en_tokens"] = doc.iloc[:, 0].apply(nlp_en.tokenizer)

In [ ]:
doc.tail()

,0,1,fr_tokens,en_tokens
57172,I really did learn a lot.,J'ai vraiment beaucoup appris.,"(J', ai, vraiment, beaucoup, appris, .)","(I, really, did, learn, a, lot, .)"
65736,That'll cost thirty euros.,Ça va faire 30 euros.,"(Ça, va, faire, 30, euros, .)","(That, 'll, cost, thirty, euros, .)"
9139,You lied to me.,Vous m'avez menti.,"(Vous, m', avez, menti, .)","(You, lied, to, me, .)"
56290,How was the French class?,Comment s'est passé le cours de français ?,"(Comment, s', est, passé, le, cours, de, franç...","(How, was, the, French, class, ?)"
88048,I excused myself for a minute.,Je m'excusai pour une minute.,"(Je, m', excusai, pour, une, minute, .)","(I, excused, myself, for, a, minute, .)"


In [ ]:
# Creation of a set() that will take all the unique tokens from our text corpus
en_tokens = [token.text for token in en_doc]
en_vocabulary_set= set(en_tokens)
en_vocab_size = len(en_vocabulary_set)
print(en_vocab_size)

3518


In [ ]:
# Same thing for French
fr_tokens = [token.text for token in fr_doc]
fr_vocabulary_set= set(fr_tokens)
fr_vocab_size = len(fr_vocabulary_set)
print(fr_vocab_size)

4903


In [ ]:
# Creation of an id for each token
all_en_tokens = {en_token: i + 1 for i, en_token in enumerate(en_vocabulary_set)}
all_fr_tokens = {fr_token: i + 1 for i, fr_token in enumerate(fr_vocabulary_set)}
# RQ: We take at i+1 to leave the value 0 for the creation of the padded_sequences

In [ ]:
# Creation of functions that will create a vector of indices for each of the token sequences
def en_tokens_to_index(tokens):
    return [all_en_tokens[token.text] for token in tokens]

def fr_tokens_to_index(tokens):
    return [all_fr_tokens[token.text] for token in tokens]

In [ ]:
# Transformation of tokens into indices
doc["fr_indices"] = doc["fr_tokens"].apply(fr_tokens_to_index)
doc["en_indices"] = doc["en_tokens"].apply(en_tokens_to_index)

In [ ]:
doc.tail()

,0,1,fr_tokens,en_tokens,fr_indices,en_indices
57172,I really did learn a lot.,J'ai vraiment beaucoup appris.,"(J', ai, vraiment, beaucoup, appris, .)","(I, really, did, learn, a, lot, .)","[3661, 3937, 1529, 4311, 1921, 922]","[2895, 499, 1626, 501, 3025, 2788, 634]"
65736,That'll cost thirty euros.,Ça va faire 30 euros.,"(Ça, va, faire, 30, euros, .)","(That, 'll, cost, thirty, euros, .)","[661, 1296, 2442, 4375, 3675, 922]","[1391, 1022, 1234, 2580, 2609, 634]"
9139,You lied to me.,Vous m'avez menti.,"(Vous, m', avez, menti, .)","(You, lied, to, me, .)","[4221, 2467, 2609, 973, 922]","[102, 3233, 2053, 923, 634]"
56290,How was the French class?,Comment s'est passé le cours de français ?,"(Comment, s', est, passé, le, cours, de, franç...","(How, was, the, French, class, ?)","[993, 1151, 3969, 241, 2553, 484, 940, 55, 157...","[1814, 640, 203, 3121, 246, 304]"
88048,I excused myself for a minute.,Je m'excusai pour une minute.,"(Je, m', excusai, pour, une, minute, .)","(I, excused, myself, for, a, minute, .)","[191, 2467, 505, 3248, 1572, 3945, 922]","[2895, 859, 3188, 3096, 3025, 2835, 634]"


In [ ]:
# Use of Keras to create token sequences of the same length
padded_fr_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["fr_indices"], padding="post")
padded_en_indices = tf.keras.preprocessing.sequence.pad_sequences(doc["en_indices"], padding="post")

In [ ]:
# Visualization of the shape of one of the tensors
padded_fr_indices.shape

(5000, 28)

In [ ]:
padded_en_indices.shape

(5000, 27)

**RQ:** the maximum length of the english sentences (28) is different from the maximum length of the french sentences (27).

In [ ]:
doc["fr_indices"].apply(len).max()

28

In [ ]:
doc["en_indices"].apply(len).max()

27

In [ ]:
# Application of the categorization of the target variable 
binarized_en_indices = tf.keras.utils.to_categorical(padded_en_indices, num_classes=en_vocab_size+1)
binarized_en_indices.shape

(5000, 27, 3519)

In [ ]:
# Creation of tf.data.Dataset for each of the French and English tensors
fr_ds = tf.data.Dataset.from_tensor_slices(padded_fr_indices)
en_ds = tf.data.Dataset.from_tensor_slices(binarized_en_indices)

In [ ]:
# Create a complete tensorflow dataset
tf_ds = tf.data.Dataset.zip((fr_ds, en_ds))

In [ ]:
next(iter(tf_ds))

(<tf.Tensor: shape=(28,), dtype=int32, numpy=
 array([2174,  384, 2031, 4493, 2756, 3969, 4389,  453,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0], dtype=int32)>,
 <tf.Tensor: shape=(27, 3519), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>)

In [ ]:
# Shuffle & Batch
BATCH_SIZE = 32

tf_ds = tf_ds.shuffle(len(doc)).batch(BATCH_SIZE)

In [ ]:
# Train Test Split
TAKE_SIZE = int(0.7 * len(doc) / BATCH_SIZE)

train_data = tf_ds.take(TAKE_SIZE)
test_data = tf_ds.skip(TAKE_SIZE)

### Modeling 

Let's move on to modeling. To create our model, we are going to use: 

* A layer of [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding?hl=en)
* 2 [LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM?hl=en) & [Bidirectional](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional?hl=en) layers
* A [RepeatVector](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RepeatVector?hl=en) layer
* A [Dense](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense?hl=en) & [TimeDistributed](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TimeDistributed?hl=en) output layer

The objective being to have in input, a tensor of dimension `(batch_size, max_len_of_french_sentences)` and in output a tensor of dimension `(batch_size, max_len_of_english_sentences, num_of_classes)` where obviously `max_len_of_english_sentences` $\neq $ `max_len_of_french_sentences`.

In [ ]:
# Create the model
model = tf.keras.Sequential([
                  # Input Word Embedding layer        
                  tf.keras.layers.Embedding(fr_vocab_size + 1, 64, mask_zero=True),

                  # LSTM Bidirectional layer
                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                  
                  # LSTM Bidirectionnal new layer
                  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),

                  # Repeat Vector
                  tf.keras.layers.RepeatVector(binarized_en_indices.shape[1]),

                  # LSTM new layer
                  tf.keras.layers.LSTM(32, return_sequences=True),               

                  # Output layer with number of output neurons equal to class number with softmax function
                  tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(en_vocab_size+1, activation="softmax"))
           
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          313856    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 27, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 27, 32)            20608     
_________________________________________________________________
time_distributed (TimeDistri (None, 27, 3519)          116127    
Total params: 615,455
Trainable params: 615,455
Non-trainable params: 0
__________________________________________________

In [ ]:
# "Random" prediction to test our model 
input_text, output_text = next(iter(train_data))
print(input_text.numpy().shape)
print(model.predict(input_text).shape)
print(output_text.numpy().shape)

(32, 28)
(32, 27, 3519)
(32, 27, 3519)


In [ ]:
# Let's create a learning rate schedule to decrease the learning rate as we train the model
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1090,
    decay_rate=0.96,
    staircase=True)

# Using a simple compiler with an Adam optimizer to compute our gradients 
optimizer= tf.keras.optimizers.Adam(
    learning_rate = lr_schedule
)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Application of the model on 200 epochs
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=200)

Epoch 1/200
109/109 [==============================] - 39s 183ms/step - loss: 6.7129 - categorical_accuracy: 0.6887 - val_loss: 2.6877 - val_categorical_accuracy: 0.7226
Epoch 2/200
109/109 [==============================] - 13s 117ms/step - loss: 2.4216 - categorical_accuracy: 0.7219 - val_loss: 2.1910 - val_categorical_accuracy: 0.7207
Epoch 3/200
109/109 [==============================] - 13s 117ms/step - loss: 2.1177 - categorical_accuracy: 0.7221 - val_loss: 1.9405 - val_categorical_accuracy: 0.7188
Epoch 4/200
109/109 [==============================] - 14s 118ms/step - loss: 1.9077 - categorical_accuracy: 0.7201 - val_loss: 1.8168 - val_categorical_accuracy: 0.7216
Epoch 5/200
109/109 [==============================] - 13s 112ms/step - loss: 1.7832 - categorical_accuracy: 0.7236 - val_loss: 1.7402 - val_categorical_accuracy: 0.7230
Epoch 6/200
109/109 [==============================] - 13s 113ms/step - loss: 1.7478 - categorical_accuracy: 0.7210 - val_loss: 1.7252 - val_categoric

In [ ]:
# Testing a translation
for input_text, translation in test_data.take(1):
    pred = np.argmax(model.predict(input_text), axis=-1)

In [ ]:
# CAUTION: it's possible because there is only one key for each value
indice_to_fr_token = dict(zip(all_fr_tokens.values(), all_fr_tokens.keys()))
indice_to_en_token = dict(zip(all_fr_tokens.values(), all_en_tokens.keys()))

In [ ]:
# French Sentence 
for indice in input_text[0]:
    if indice == 0:
        break
    print(indice_to_fr_token[indice.numpy()])

Tom
savait
que
Mary
comprenait
ce
qu'
ils
avaient
besoin
de
faire
.


In [ ]:
# Real English Sentence 
for indice in np.argmax(translation, axis=-1)[0]:
    if indice == 0:
        break
    print(indice_to_en_token[indice])

Tom
knew
Mary
understood
what
they
needed
to
do
.


In [ ]:
# Sentence translated into English by the model 
for indice in pred[0]:
    if indice == 0:
        break
    print(indice_to_en_token[indice])

Tom
did
Mary
he
what
what
he
to
do
.


In [ ]:
# Training on 500 more epochs
history_2 = model.fit(train_data,
                      validation_data=test_data,
                      epochs=500)

Epoch 1/500
109/109 [==============================] - 12s 108ms/step - loss: 0.8248 - categorical_accuracy: 0.8354 - val_loss: 0.8245 - val_categorical_accuracy: 0.8351
Epoch 2/500
109/109 [==============================] - 13s 109ms/step - loss: 0.8419 - categorical_accuracy: 0.8324 - val_loss: 0.8352 - val_categorical_accuracy: 0.8333
Epoch 3/500
109/109 [==============================] - 13s 111ms/step - loss: 0.8330 - categorical_accuracy: 0.8338 - val_loss: 0.8349 - val_categorical_accuracy: 0.8336
Epoch 4/500
109/109 [==============================] - 13s 109ms/step - loss: 0.8395 - categorical_accuracy: 0.8327 - val_loss: 0.8368 - val_categorical_accuracy: 0.8326
Epoch 5/500
109/109 [==============================] - 13s 110ms/step - loss: 0.8259 - categorical_accuracy: 0.8351 - val_loss: 0.8258 - val_categorical_accuracy: 0.8355
Epoch 6/500
109/109 [==============================] - 13s 109ms/step - loss: 0.8354 - categorical_accuracy: 0.8330 - val_loss: 0.8259 - val_categoric

In [ ]:
# Testing on new translations

for input_text, translation in test_data.take(1):
    pred = np.argmax(model.predict(input_text), axis=-1)

# French 
print("Input Sentence:", end=" ")
for indice in input_text[2]:
    if indice == 0:
        break
    print(indice_to_fr_token[indice.numpy()], end=" ")

# True
print("\nTrue Translation:", end=" ")
for indice in np.argmax(translation, axis=-1)[2]:
    if indice == 0:
        break
    print(indice_to_en_token[indice], end=" ")

# Pred
print("\nModel Translation:", end=" ")
for indice in pred[2]:
    if indice == 0:
        break
    print(indice_to_en_token[indice], end=" ")

Input Sentence: L' argent ne nous rend pas nécessairement plus heureux . 
True Translation: Money does n't necessarily make you happier . 
Model Translation: Money did n't necessarily to you happier . 

After 2 training steps, the model reached an accuracy of 0.89 on the validation set. Nevertheless, we still notice some weaknesses guessing the time used or translating the verbal group.

A solution may be first to run the model on the whole dataset to increase the amount of training data, and then adapt the number of layers in our neural network.